In [11]:
import numpy as np

$y = \sin(\exp(x^3))$

$\frac{dy}{dx} = \cos(\exp(x^3)) \exp(x^3) 3 x^2$

In [27]:
class Variable:

    def __init__(self, name, initial_value):
        self.name = name
        self.value = initial_value
        self.grad = 0.0

    def evaluate(self):
        return self.value

    def zero_grad(self):
        self.grad = 0.0

    def backward(self, upstream_grad):
        self.grad += upstream_grad

    def __str__(self):
        return f'{self.name}:{self.value}'


class Cube:

    def __init__(self, x):
        self.x = x

    def evaluate(self):
        return self.x.evaluate()**3

    def backward(self, upstream_grad):
        local_grad = 3 * self.x.evaluate()**2
        downstream_grad = upstream_grad * local_grad
        self.x.backward(downstream_grad)

    def __str__(self):
        return f'({self.x})^3'


class Exp:

    def __init__(self, x):
        self.x = x

    def evaluate(self):
        return np.exp(self.x.evaluate())

    def backward(self, upstream_grad):
        local_grad = np.exp(self.x.evaluate())
        downstream_grad = upstream_grad * local_grad
        self.x.backward(downstream_grad)

    def __str__(self):
        return f'exp({self.x})'


class Sin:

    def __init__(self, x):
        self.x = x

    def evaluate(self):
        return np.sin(self.x.evaluate())

    def backward(self, upstream_grad):
        local_grad = np.cos(self.x.evaluate())
        downstream_grad = upstream_grad * local_grad
        self.x.backward(downstream_grad)

    def __str__(self):
        return f'sin({self.x})'

In [30]:
x = Variable('x', 2)
c = Cube(x)
e = Exp(c)
y = Sin(e)

print(y.evaluate())

x.zero_grad()
y.backward(1.0)

print(x.grad)


0.40176297151886475
-32757.521776103014


In [31]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape((x_train.shape[0], 28 * 28)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28 * 28)).astype('float32') / 255

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create the MLP model
model = Sequential()
model.add(Flatten(input_shape=(28 * 28,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy}')

ModuleNotFoundError: No module named 'tensorflow'